In [38]:
import pandas as pd
import numpy as np

import import_ipynb
import preprocessamentofuncoes as pp

importing Jupyter notebook from preprocessamentofuncoes.ipynb


In [3]:
# Carregando as bases
parte_1 = pd.read_excel('PreProcessamento/sus-sp1e4-pp.xlsx')
#parte_2 = pd.read_excel('PreProcessamento/sus-sp5e7-pp.xlsx')

In [ ]:
# Juntando as partes
# covidData = pd.concat([parte_1, parte_2])
# covidData.reset_index(inplace=True, drop = True)

In [219]:
covidData = parte_1
covidData.shape

(486301, 49)

In [220]:
covidData.columns

Index(['source_id', 'dataInicioSintomas', 'dataNotificacao', 'sexo', 'racaCor',
       'idade', 'sintomas', 'outrosSintomas', 'condicoes', 'outrasCondicoes',
       'dataPrimeiraDose', 'dataSegundaDose', 'classificacaoFinal',
       'evolucaoCaso', 'diasSintomas', 'assintomatico', 'febre',
       'dorDeGarganta', 'dispneia', 'tosse', 'coriza', 'dorDeCabeca',
       'disturbiosGustatorios', 'disturbiosOlfativos', 'mialgia', 'dorNoCorpo',
       'diarreia', 'cansaco', 'dosNasCostas', 'congestaoNasal', 'vomito',
       'dorNoPeito', 'faltaDeAr', 'cefaleia', 'diabetes', 'obesidade', 'renal',
       'respiratoria', 'imunossupressao', 'fragilidadeImuno', 'gestante',
       'cardiaca', 'puerpera', 'hipertensao', 'asma', 'bronquite',
       'hipotireoidismo', 'qntVacinas', 'faixaetaria'],
      dtype='object')

In [221]:
colunasRemovidas = ['source_id', 'dataInicioSintomas', 'dataNotificacao', 'idade',
                    'sintomas', 'outrosSintomas', 'condicoes', 'outrasCondicoes',
                    'dataPrimeiraDose', 'dataSegundaDose', 'classificacaoFinal', 
                    'outrosSintomas', 'outrasCondicoes']

covidData = covidData.drop(columns = colunasRemovidas)
covidData.shape

(486301, 38)

### Verificando os atributos

In [13]:
faltantes = covidData.isnull().sum()
faltantes_percentual = (covidData.isnull().sum() / len(covidData)) * 100
faltantes_percentual

sexo                     0.000000
racaCor                  0.000000
evolucaoCaso             0.000000
diasSintomas             6.823552
assintomatico            0.000000
febre                    0.000000
dorDeGarganta            0.000000
dispneia                 0.000000
tosse                    0.000000
coriza                   0.000000
dorDeCabeca              0.000000
disturbiosGustatorios    0.000000
disturbiosOlfativos      0.000000
mialgia                  0.000000
dorNoCorpo               0.000000
diarreia                 0.000000
cansaco                  0.000000
dosNasCostas             0.000000
congestaoNasal           0.000000
vomito                   0.000000
dorNoPeito               0.000000
faltaDeAr                0.000000
cefaleia                 0.000000
diabetes                 0.000000
obesidade                0.000000
renal                    0.000000
respiratoria             0.000000
imunossupressao          0.000000
fragilidadeImuno         0.000000
gestante      

In [222]:
# Excluindo os dias vazios
diasVazios = covidData.loc[covidData['diasSintomas'].isnull()]
covidData = covidData.drop(diasVazios.index)
covidData.shape

(453118, 38)

In [224]:
# Verificando a distribuição de valores em cada atributo
# colunas = covidData.columns
# for coluna in colunas:
#     print(covidData[coluna].value_counts())

In [225]:
covidData['evolucaoCaso'].value_counts()

1    451814
0       905
2       399
Name: evolucaoCaso, dtype: int64

In [226]:
# Removendo os dias outliers
diasSintomasOutliers = covidData.loc[((covidData['diasSintomas'] > 30))]
covidData = covidData.drop(diasSintomasOutliers.index)
covidData['evolucaoCaso'].value_counts()

1    430776
0       863
2       380
Name: evolucaoCaso, dtype: int64

In [227]:
def converterDiasSintomas(covidData, diasSintomas, novaColuna):
    covidData.loc[((covidData[diasSintomas] >= 0) & (covidData[diasSintomas] <= 3)), novaColuna] = 0
    covidData.loc[((covidData[diasSintomas] >= 4) & (covidData[diasSintomas] <= 6)), novaColuna] = 1
    covidData.loc[((covidData[diasSintomas] >= 7) & (covidData[diasSintomas] <= 9)), novaColuna] = 2
    covidData.loc[((covidData[diasSintomas] >= 10) & (covidData[diasSintomas] <= 12)), novaColuna] = 3
    covidData.loc[((covidData[diasSintomas] >= 13) & (covidData[diasSintomas] <= 15)), novaColuna] = 4
    covidData.loc[((covidData[diasSintomas] >= 16) & (covidData[diasSintomas] <= 18)), novaColuna] = 5
    covidData.loc[((covidData[diasSintomas] > 18)), novaColuna] = 6
    return covidData

In [228]:
covidData = converterDiasSintomas(covidData, 'diasSintomas', 'faixaDiasSintomas')

In [230]:
covidData['faixaDiasSintomas'].value_counts()

1.0    192837
0.0    147705
2.0     49546
3.0     16591
6.0     11540
4.0      8208
5.0      5533
Name: faixaDiasSintomas, dtype: int64

### Balanceamento dos Dados

In [284]:
covidDataBalanced = pp.balanceamento(covidData, 'evolucaoCaso', 2, 30, 1)
covidDataBalanced.shape

(1646, 39)

In [285]:
covidDataBalanced['evolucaoCaso'].value_counts()

1    1266
2     380
Name: evolucaoCaso, dtype: int64

In [281]:
# Salvando os Dados PreProcessados
write = pd.ExcelWriter('PreProcessamento/balanceadas/new-pp-ic-7030-sp.xlsx')
covidDataBalanced.to_excel(write, 'dados', index = False)
write.save()

### Testando o Desempenho do Modelo

In [286]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

from sklearn.ensemble import GradientBoostingClassifier


In [287]:
hiperparametrosLR = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, 
                                       intercept_scaling=1, l1_ratio=None, max_iter=1000, multi_class='auto', 
                                       n_jobs=None, penalty='l2', random_state=1340, solver='lbfgs', 
                                       tol=0.0001, verbose=0, warm_start=False)

hiperparametrosGB = GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, 
                           random_state=2333, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [291]:
atributosRemovidos = ['assintomatico', 'hipotireoidismo', 'vomito', 'imunossupressao', 
                    'cefaleia', 'puerpera', 'disturbiosGustatorios', 'congestaoNasal', 
                    'bronquite', 'disturbiosOlfativos', 'cansaco', 'asma', 'diasSintomas',]

covidDataBalanced = covidDataBalanced.drop(columns = atributosRemovidos)
covidDataBalanced.shape

(1646, 26)

In [292]:
baseObitoCurado = covidDataBalanced
alvo = 'evolucaoCaso'
x_train, x_test, y_train, y_test = train_test_split(baseObitoCurado.drop(alvo,axis=1),
                                                    baseObitoCurado[alvo],
                                                    test_size=0.3,
                                                    random_state=42);

In [293]:
# Fit nos dados
modeloClassificador = hiperparametrosGB
modeloClassificador.fit(x_train, y_train)

#Realizando a predição
resultadoTeste = modeloClassificador.predict(x_test)

# Verificando a acurácia
print(metrics.classification_report(y_test,resultadoTeste))

              precision    recall  f1-score   support

           1       0.84      0.94      0.89       383
           2       0.66      0.38      0.48       111

    accuracy                           0.82       494
   macro avg       0.75      0.66      0.68       494
weighted avg       0.80      0.82      0.80       494



In [294]:
feature_importances = pd.DataFrame(modeloClassificador.feature_importances_,
                                   index = x_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
faixaetaria,0.474973
dispneia,0.099109
dorDeCabeca,0.095416
faixaDiasSintomas,0.057907
dorDeGarganta,0.046485
coriza,0.026309
renal,0.025820
diabetes,0.023813
respiratoria,0.021592
racaCor,0.018782


### Colunas - Modelo OC

In [283]:
colunasRemovidas = ['assintomatico', 'hipotireoidismo', 'vomito', 'imunossupressao', 
                    'cefaleia', 'puerpera', 'disturbiosGustatorios', 'congestaoNasal', 
                    'bronquite', 'disturbiosOlfativos', 'cansaco', 'asma', 'diasSintomas',]

colunasRemovidas = ['assintomatico', 'hipotireoidismo', 'vomito', 'imunossupressao', 
                    'cefaleia', 'puerpera', 'disturbiosGustatorios', 'congestaoNasal', 
                    'bronquite', 'disturbiosOlfativos', 'cansaco', 'asma', 'diasSintomas', 
                    'tosse', 'gestante', 'faltaDeAr', 'mialgia', 'diarreia', 
                    'dosNasCostas', 'fragilidadeImuno', 'respiratoria', 'dorNoPeito',]

# Menos de 1% (0.8 - 0.2)
colunasRemovidas = ['assintomatico', 'hipotireoidismo', 'vomito', 'imunossupressao', 
                    'cefaleia', 'puerpera', 'disturbiosGustatorios', 'congestaoNasal', 
                    'bronquite', 'disturbiosOlfativos', 'cansaco', 'asma', 'diasSintomas', 
                    'tosse', 'gestante', 'faltaDeAr', 'mialgia', 'diarreia', 
                    'dosNasCostas', 'fragilidadeImuno', 'respiratoria', 'dorNoPeito',
                    'dorDeCabeca', 'dorNoCorpo', 'obesidade', 'febre', 'renal', 'hipertensao',]

### Colunas - Modelo IC